In [19]:
import requests
import datetime
import time
import pytz
import csv
import warnings
from dateutil import parser
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from urllib3.exceptions import InsecureRequestWarning
from collections import defaultdict 

# Disable urllib3 InsecureRequestWarning
warnings.filterwarnings('ignore', category=InsecureRequestWarning)

from datetime import date, timedelta

class APIReader:

    @staticmethod
    def create_session():
        session = requests.Session()
        retries = Retry(total=5, 
                        backoff_factor=1, 
                        status_forcelist=[429, 500, 502, 503, 504],
                        allowed_methods=["HEAD", "GET", "OPTIONS"])
        session.mount("https://", HTTPAdapter(max_retries=retries))
        return session


    def __init__(self):
        APIReader.self = self


    def return_daylight():

        # Set timezone to Central European Time
        serbia_tz = pytz.timezone("Europe/Belgrade")

        # Get current time in Serbia
        current_time = datetime.datetime.now(serbia_tz)

        # Determine whether daily saving time is in effect in order to set adjustment
        daylight_saving = current_time.dst() != timedelta(0)
        return daylight_saving


    @staticmethod
    def collect_data(day):
        session = APIReader.create_session()

        yesterday = (datetime.datetime.strptime(day, '%Y-%m-%d') - timedelta(days=1)).strftime('%Y-%m-%d')
        tomorrow = (datetime.datetime.strptime(day, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')

        site_urlcon = "https://ems.energyflux.rs/"
        site_urlprod = "https://ems.energyflux.rs/"
        api_urlcon = f"{site_urlcon}api/consumption/SRBIJA?fromDate={yesterday}T22:00:00.000Z&toDate={tomorrow}T21:59:59.999Z"
        api_urlprod = f"{site_urlprod}api/production/SRBIJA?sourceType=NONE&fromDate={yesterday}T22:00:00.000Z&toDate={tomorrow}T21:59:59.999Z"

        max_retries = 5
        for attempt in range(max_retries):
            try:
                response_c = session.get(api_urlcon, verify=False, timeout=30)
                response_c.raise_for_status()
                data_c = response_c.json()

                response_p = session.get(api_urlprod, verify=False, timeout=30)
                response_p.raise_for_status()
                data_p = response_p.json()

                # Process data_c and data_p as before
                energy_p = {}
                energy_c = {}
                
                if "production" in data_p:
                    production_data = data_p["production"]
                    for source, values in production_data.items():
                        energy_p[source] = {item["t"]: item["v"] for item in values}
                
                if "consumption" in data_c:
                    consumption_data = data_c["consumption"]
                    energy_c = {item["t"]: item["v"] for item in consumption_data}

                return [energy_c, energy_p]

            except (requests.exceptions.RequestException, ValueError) as e:
                if attempt < max_retries - 1:
                    wait_time = 2 ** attempt  # exponential backoff
                    print(f"Error occurred: {e}. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    print(f"Failed to fetch data for {day} after {max_retries} attempts.")
                    return None

    def print_data():
        # Print today's data for prices within Serbian exchange
        today = str(datetime.datetime.today()).split()[0]
        combined_data = APIReader.collect_data(today)

        print("\n")
        print("Serbian Production/Consumption: \n")
        
        energy_c, energy_p = combined_data
        
        # Get all unique timepoints
        timepoints = set(energy_c.keys())
        for source in energy_p.values():
            timepoints.update(source.keys())
        
        for timepoint in sorted(timepoints):
            consumption = energy_c.get(timepoint, "N/A")
            
            production = {}
            for source, values in energy_p.items():
                production[source] = values.get(timepoint, "N/A")
            
            # Parse the timepoint string to a datetime object
            try:
                local_time = parser.parse(timepoint)
                # If the parsed time is naive (no timezone info), assume it's in UTC
                if local_time.tzinfo is None:
                    local_time = pytz.utc.localize(local_time)
                # Convert to Belgrade time
                local_time = local_time.astimezone(pytz.timezone("Europe/Belgrade"))
            except ValueError:
                print(f"Unable to parse time: {timepoint}")
                continue
            
            print(f"Date: {local_time.strftime('%Y-%m-%d')}  Time: {local_time.strftime('%H:%M')}")
            print(f"  Consumption: {consumption}")
            for source, value in production.items():
                print(f"  Production ({source}): {value}")
            print()



    @staticmethod
    def collect_historical_data(start_date, end_date):
        all_data = []
        current_date = start_date
        while current_date <= end_date:
            print(f"Collecting data for {current_date}")
            day_data = APIReader.collect_data(current_date.strftime('%Y-%m-%d'))
            if day_data is not None:
                all_data.append((current_date, day_data))
            else:
                print(f"Skipping {current_date} due to data collection failure.")
            current_date += timedelta(days=1)
            time.sleep(1)  # Add a delay between requests to avoid overwhelming the server
        return all_data

    @staticmethod
    def save_to_csv(data, filename):
        with open(filename, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            
            # Get the production sources from the first non-empty data point
            production_sources = []
            for _, (_, energy_p) in data:
                if energy_p:
                    production_sources = list(energy_p.keys())
                    break
            
            # Write header
            csvwriter.writerow(['Date', 'Time', 'Consumption'] + [f'Production ({source})' for source in production_sources])
            
            for date, (energy_c, energy_p) in data:
                timepoints = set(energy_c.keys())
                for source in energy_p.values():
                    timepoints.update(source.keys())
                
                for timepoint in sorted(timepoints):
                    try:
                        local_time = parser.parse(timepoint)
                        if local_time.tzinfo is None:
                            local_time = pytz.utc.localize(local_time)
                        local_time = local_time.astimezone(pytz.timezone("Europe/Belgrade"))
                        
                        # Only process timepoints for the current date
                        if local_time.date() == date:
                            consumption = energy_c.get(timepoint, "N/A")
                            production = [energy_p.get(source, {}).get(timepoint, "N/A") for source in production_sources]
                            
                            csvwriter.writerow([local_time.strftime('%Y-%m-%d'), local_time.strftime('%H:%M'), consumption] + production)
                    except ValueError:
                        print(f"Unable to parse time: {timepoint}")
                        continue

    @staticmethod
    def process_historical_data(start_date, end_date):
        if start_date > end_date:
            print("Error: Start date must be earlier than or equal to end date.")
            return

        print(f"Collecting data from {start_date} to {end_date}")
        historical_data = APIReader.collect_historical_data(start_date, end_date)
        
        filename = f'serbian_energy_data_{start_date.strftime("%Y%m%d")}_to_{end_date.strftime("%Y%m%d")}.csv'
        APIReader.save_to_csv(historical_data, filename)

        print(f"Data collection completed. Data saved to {filename}")

        # Check for missing dates and timepoints
        APIReader.check_data_completeness(historical_data, start_date, end_date)

    @staticmethod
    def check_data_completeness(data, start_date, end_date):
        collected_dates = set(date for date, _ in data)
        all_dates = set(start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1))
        
        missing_dates = all_dates - collected_dates
        if missing_dates:
            print("\nMissing dates:")
            for date in sorted(missing_dates):
                print(date.strftime("%Y-%m-%d"))
        else:
            print("\nNo missing dates.")

        missing_timepoints = defaultdict(list)
        for date, (energy_c, energy_p) in data:
            timepoints = set(energy_c.keys())
            for source in energy_p.values():
                timepoints.update(source.keys())
            
            if len(timepoints) < 144:
                missing_count = 144 - len(timepoints)
                missing_timepoints[date].append(f"{missing_count} timepoints")

        if missing_timepoints:
            print("\nDates with missing timepoints:")
            for date, missing in missing_timepoints.items():
                print(f"{date.strftime('%Y-%m-%d')}: {', '.join(missing)}")
        else:
            print("\nNo missing timepoints in any day.")

start_date = datetime.date(2020, 2, 12)
end_date = datetime.date(2024, 7, 28)
APIReader.process_historical_data(start_date, end_date)


Data collection completed. Data saved to serbian_energy_data_20200212_to_20240728.csv

No missing dates.

Dates with missing timepoints:
2021-07-25: 27 timepoints
2024-07-28: 117 timepoints
